## Tutorial 4: Canonical Forms

### As found in [here](https://www.tensors.net/tutorial-4)

In this (last?) tutorial we shall build upon and extend some of the ideas introduced in Tutorial 3, including how to properly compose multiple tensor decompositions as well as more sophisticated means to fix the gauge degrees of freedom, leading to the notion of a canonical form for TN. Topics include:

* Multi-stage tensor decompositions
* Centers of orthogonality on tensor links
* Canonical forms of TN

In [2]:
import numpy as np
from numpy import linalg as LA
from ncon import ncon

### Multi-stage tensor decompositions

We begin by addressing an important problem: given a many-index tensor $H$, how can we accurately decompose this into a network of $T$ tensors $\{A, B, C, ... \}$ according to some prescribed geometry, for instance, as depicted below? More precisely, we'd like to find a choice of tensors that minimizes the difference $|| H - T ||$ from the original tensor, given some fixed dimension $\chi$ of the internal indices in $T$.

<img src="img/fig1.png" alt="drawing" width="500"/>

In order to obtain the network approximation to a tensor we shall employ a multi-stage decomposition: a sequence of single tensor decompositions via the SVD. The results from tutorial 3 already inform us of the correct way to perform a multi-stage decomposition: the tensor to be decomposed at each step should be a center of orthogonality, which will ensure that the global truncation error is minimized.

The figure below illustrates a sequence of single tensor decompositions that take a single tensor $H_0$ into the network $T$ of the previous figure. At each step, a tensor $H_k$ is split using a truncated SVD (retaining the desired rank $\chi$) into a product of three tensors $\{U_k, S_k, V_k \}$ across the partition indicated by the dashed line, where we have coloured isometric tensors orange. The matrix of singular values $S_k$ is then absorbed into the tensor that is to be decomposed at the next step, such that it becomes a center of orthogonality. This process is repeated until the desired network geometry is reached.

<img src="img/fig2.png" alt="drawing" width="700"/>



In [3]:
##### Example code: multi-stage decomposition

d = 5 # local dimension
chi = 3 # max internal dimension 
H0 = (np.sqrt(1+np.arange(d**7))).reshape(d,d,d,d,d,d,d).transpose(6,5,4,3,2,1,0)

# first decomposition
utemp,stemp,vhtemp = LA.svd(H0.reshape(d**2,d**5),full_matrices=False)
U0 = (utemp[:,:chi]).reshape(d,d,chi)
H1 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d,d,d)

# second decomposition
utemp,stemp,vhtemp = LA.svd(H1.transpose(1,2,0,3,4,5).reshape(d**2,chi*d**3),full_matrices=False)
U1 = (utemp[:,:chi]).reshape(d,d,chi)
H2 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,chi,d,d,d).transpose(1,0,2,3,4)

# third decomposition
utemp,stemp,vhtemp = LA.svd(H2.reshape(chi**2,d**3),full_matrices=False)
U2 = (utemp[:,:chi]).reshape(chi,chi,chi)
H3 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d)

# fourth decomposition
utemp,stemp,vhtemp = LA.svd(H3.reshape(chi*d,d**2),full_matrices=False)
V3 = vhtemp[:chi,:].reshape(chi,d,d).transpose(1,2,0)
H4 = (utemp[:,:chi] @ np.diag(stemp[:chi])).reshape(chi,d,chi)

# check result
H0recovered = ncon([U0,U1,U2,V3,H4],[[-1,-2,1],[-3,-4,2],[1,2,3],[-6,-7,4],[3,-5,4]])
totErr = LA.norm(H0 - H0recovered) / LA.norm(H0)

print(totErr)

6.381973359135423e-05


**Notes on multi-stage decompositions:**

* One has freedom in choosing the sequence of decompositions in a multi-stage decomposition. If the decompositions include index truncations, then some sequences could be computationally cheaper than others.

* When following the outlined procedure for multi-stage decompositions the truncation error at each individual decomposition step is minimized. However the cumulative error from the sequence of decompositions is not guaranteed to be minimal, and may indeed change depending on the particular sequence of decompositions used.

### Center of orthogonality (link centered)

We extend the idea of center of orthogonality to links in a TN.

The idea is that one can introduce a new link matrix $\sigma$ situated on the link in question, which is initially defined as the identity matrix, and then proceed using one of the previous methods to fix $\sigma$ as a center of orthgonality. This is illustrated in the figure below, where the direct orthogonalization approach is used to set the $B-C$ link as a center of orthgonality.

<img src="img/tut4_fig3.png" alt="drawing" width="700"/>

For the link-centered center of orthogonality we shall impose an additional constraint: that the gauge shoukd be chosen such that the final form of link matrix should be diagonal, with positive elements in descending order of magnitude. 

Achieving this requires a final step over the previous orthogonalization approaches, where one should take the SVD of the link matrix $\sigma'$ and then make a unitary change of gauge such that it is brought into diagonal form. The next figure illustrates this procedure, continuing on from the example in the previous figure.

<img src="img/tut4_fig4.png" alt="drawing" width="700"/>

In [15]:
##### Example code: set B-C link as center of orthogonality

d = 5 # index dimension
A = np.random.rand(d,d,d)
B = np.random.rand(d,d,d)
C = np.random.rand(d,d,d)
Sig = np.eye(d); # initial link matrix = identity matrix

# generate gauge change matrices
rho1 = ncon([A,A,B,B],[[1,2,3],[1,2,4],[3,5,-1],[4,5,-2]])
rho2 = ncon([C,C],[[-1,1,2],[-2,1,2]])
d1, u1 = LA.eigh(rho1); sq_d1 = np.sqrt(abs(d1))
d2, u2 = LA.eigh(rho2); sq_d2 = np.sqrt(abs(d2))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T

# implement gauge change
Bprime = ncon([B,X1inv],[[-1,-2,1],[1,-3]])
Cprime = ncon([X2inv,C],[[-1,1],[1,-2,-3]])
Sig_prime = X1 @ Sig @ X2

# perform unitary gauge change to diagonalize link matrix
utemp, Sig_pp, vhtemp = LA.svd(Sig_prime,full_matrices=False)
Bpp = ncon([Bprime,utemp],[[-1,-2,1],[1,-3]])
Cpp = ncon([Cprime,vhtemp],[[1,-2,-3],[-1,1]])

# check result
H1 = ncon([A,Bpp,np.diag(Sig_pp),Cpp],[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]])
totErr = LA.norm(H0 - H1) / LA.norm(H0) 

print(totErr)

0.5032603295021318


#### Equivalence of the (link) center of orthogonality and the SVD:

Given a network of tensors that contracts to a single tensor $H$, the form of the network produced from fixing a link as the center of orthogonality is related to an SVD of the tensor $H$, as we now explain.

Let's continue with last example. We define tensor $H$ aas the result of contracting the original network and assume the SVD produces $H = U_H \cdot S_H \cdot V_H^\dagger $. However, the network produced from setting tensor $\sigma''$ as a center of orthogonality is understood as equivalent to an SVD given:

(i) The grouping of network to the left of the center forms an isometry

(ii) Matrix $\sigma''$ is diagonal with positive elements in ascending order

(iii) The network to the right of the center is also an isometry.

Assuming that the singular values are non-degenerate, the SVD is known to be unique up to the signs of the singular vectors (or phases, for complex matrices). Thus we conclude that $U_H = A \cdot B''$ and $V_H^\dagger = C''$, up to signs/phases, while the link matrix $\sigma''$ precisely equals the matrix of singular values $S_H$.

In summary, the result of fixing a link within the network as a center of orthogonality is equivalent to an SVD: the resulting link matrix $\sigma''$ contains the singular weights, while the networks to the left/right are equivalent under contraction to the U/V isometries.

<img src="img/tut4_fig6.png" alt="drawing" width="700"/>



In [16]:
##### Example code: equivalence of center of orthogonality and SVD
H = ncon([A,B,C],[[-1,-2,1],[1,-3,2],[2,-4,-5]])
utemp,stemp,vhtemp = LA.svd(H.reshape(d**3,d**2),full_matrices=False)
UH = utemp[:,:d].reshape(d,d,d,d)
SH = stemp[:d]
VH = vhtemp[:d,:].reshape(d,d,d)

# compare with previous tensors from orthonormal form 
ErrU = LA.norm(abs(UH) - abs(ncon([A,Bpp],[[-1,-2,1],[1,-3,-4]])))
ErrS = LA.norm(SH - Sig_pp) 
ErrV = LA.norm(abs(VH) - abs(Cpp))
# all three results should be vanishingly small!!!

print(ErrU)
print(ErrS)
print(ErrV)

5.23595037859644e-14
3.285359848735979e-14
5.076512510981851e-14


The identification between setting a center of orthogonality at a network link and the SVD of the network across the corresponding partition is a particularly useful result. Foremost, this provides a convenient means to optimally truncate indices within a network, as outlined in the following theorem.

**Theorem**: Given a (loop-free) network of tensors $\{A, B, C, ...\}$, assume that we wish to truncate the link between a pair of tensors to a reduced dimension. Then the optimal truncation is given by first transforming the link in question into a center of orthogonality, and then truncating the smallest singular values from the corresponding link matrix.  The truncation error is given as the square-root of the sum of the squares of the discarded singular values.

## Canonical forms

As discussed in tutorial 3, given a network of tensors $\{A, B, C, ... \}$ we can manipulate the gauge degrees of freedom to set a chosen tensor $A$ as a center of orthogonality. However, this does not fully fix the gauge degree of freedom on any of the network links; the tensor $A$ will still remain a center of orthogonality under a unitary change of gauge.

In contrast, when setting a link as a center of orthogonality, the additional constraint that the link matrix is diagonal with positive elements in ascending order can completely fix the gauge freedom of this link. In general, this is true if the elements of the link matrix are non-degenerate, otherwise there will remain a unitary gauge freedom within the degenerate subspace. This uniqueness property follows from the correspondence between the link-based center of orthogonality and the SVD, which is similarly unique. 

In this section we describe a method for simultaneously setting every link of a network as a center of orthogonality, which also dictates a unique way to fix the gauge on each of the network links (i.e. such that no further non-trivial gauge changes can be made without destroying at least one of the centers of orthogonality). A network in this form, with every link simultaneously a center of orthogonality, is said to be in canonical form (or sometimes normal form).

#### Procedure for transforming a network to canonical form:

* Position a link matrix $\sigma$, initially set as the identity matrix, on each internal index.

* Use the direct orthogonalization approach to independently set every link matrix $\sigma$ as a center, i.e. by making a gauge change of the form $X$ and $X^{-1}$ on each link, where $X$ is the principal square root of the corresponding branch density matrix $\rho$.

* Make a unitary change of gauge on each link, with unitary matrices $U$ and $V$ given from the SVD of the link matrix $\sigma'$ and their conjugates, such that the link matrix is transformed into a diagonal matrix with positive elements in ascending order.

<img src="img/tut4_fig7.png" alt="drawing" width="700"/>

A network that is in canonical form, such that all link matrices are simultaneously centers of orthogonality, can be manipulated in a number of useful ways. In particular, we can easily fix any chosen tensor as a center of orthogonality, thus allowing us to recover the results from tutorial 3. 

Consider, for example, the network in the figure below, which is presumed to be in canonical form. (i) Assuming we wish to set some specified tensor $E$ as the center of orthogonality, we first label each index with an arrow that points towards tensor E. (ii) We then absorb into each tensor any link matrices $\sigma$ that reside on its incoming indices. (iii) It follows from properties of the canonical form that all tensors in the resulting network are now isometric with respect to the partition between their incoming and outgoing tensor indices, similar to the network produced from the 'pulling through' approach. Moreover, the specified tensor $E$ is a center of orthogonality in agreement with the established definition.

<img src="img/tut4_fig8.png" alt="drawing" width="700"/>


### Summary: Canonical Forms

The canonical form results from a particular way of fixing the gauge degrees of freedom in a tensor network, and is useful as it organizes a network in such a way that important information is readily available. By setting a network in canonical form one can:

* Optimally truncate any link to reduced dimension, simply discarding the smallest singular values from the corresponding link matrix, and easily understand the corresponding truncation error.

* Essentially remove the gauge ambiguity from the network description, due to the uniqueness of the canonical form.

* Extract certain types information from the network (such as expectation values) in a simplified manner, due to the cancellations that arise in networks containing isometric tensors.

----

### Problem Set 4:

Consider the tensor $H$ with the element-wise definition given by

$$ H_{ijklm} = \sqrt{i+2j+3k+4l+5m+15} \quad \text{(Python)}$$

**(a)** Generate tensor $H$ assuming all indices are dimension $d = 6$ and evaluate the norm $||H||$. Define the normalized tensor $H_1 = H / ||H||$.

In [17]:
# (a) Define H and normalize
d = 6

H = np.zeros((d,d,d,d,d))

for i in range(d):
    for j in range(d):
        for k in range(d):
            for l in range(d):
                for m in range(d):
                    H[i,j,k,l,m] = np.sqrt(i + 2.*j + 3.*k + 4.*l + 5.*m + 15)
                    
H1 = H / LA.norm(H)

**(b)** Use a multi-stage decomposition to split $H_1$ into a network of three tensors $\{A, B, C\}$ while truncating to index dimension $\chi = 6$, according to the following figure:

<img src="img/tut4_fig10.png" alt="drawing" width="500"/>

Compute the truncation error $\varepsilon = ||H_1 - H_2||$, where $H_2$ is the tensor recovered from contracting $\{A, B, C\}$.

In [21]:
# (b) Split H1 using a multi-stage decomposition and truncate to index dimension chi=6
chi = 6

# first decomposition
utemp,stemp,vhtemp = LA.svd(H1.reshape(d**2,d**3),full_matrices=False)
A = (utemp[:,:chi]).reshape(d,d,chi)
H2 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d)

# second decomposition
utemp,stemp,vhtemp = LA.svd(H2.reshape(chi*d,d**2),full_matrices=False)
B = (utemp[:,:chi]).reshape(chi,d,chi)
C = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d)

# check result
H1recovered = ncon([A,B,C],[[-1,-2,1],[1,-3,2],[2,-4,-5]])
totErr = LA.norm(H1 - H1recovered)

print(totErr)

9.504156572584146e-11


**(c)** Make the appropriate change of gauge sich that the $A-B$ link becomes a center of orthogonality with a diagonal matrix $\sigma'$. What are the singular values in $\sigma'$? Check the validity of your gauge transformation by confirming that the singular values in $\sigma'$ match those obtained directly from the SVD of tensor $H_2$ across the equivalent partition.

In [25]:
# (c) Make link A-B a center. Compute singular values of sigmap. Check validity of gauge transf

Sig = np.eye(chi); # initial link matrix = identity matrix

# generate gauge change matrices
rho1 = ncon([A,A],[[1,2,-1],[1,2,-2]])
rho2 = ncon([B,B,C,C],[[-1,1,2],[-2,1,3],[2,4,5],[3,4,5]])
d1, u1 = LA.eigh(rho1); sq_d1 = np.sqrt(abs(d1))
d2, u2 = LA.eigh(rho2); sq_d2 = np.sqrt(abs(d2))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T


utemp,Sigp,vhtemp = LA.svd(X1 @ Sig @ X2)
Ap = ncon([A,X1inv*utemp],[[-1,-2,1],[1,-3]])
Bp = ncon([B,vhtemp*X2inv],[[1,-2,-3],[-1,1]])

# difference in singular values between SVD:
_,svalues,_ = LA.svd(H1recovered.reshape(d**2,d**3),full_matrices=False)
Serr = LA.norm(svalues[:chi] - Sigp)

print(Serr)
print(svalues)

4.459963872560997e-16
[9.99958526e-01 9.10710202e-03 7.97572086e-05 1.61449112e-06
 3.64748531e-08 8.06121453e-10 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17
 9.98939531e-17 9.98939531e-17 9.98939531e-17 9.98939531e-17]


**(d)** Assume that the network $T$ of the following figure is in canonical form:

<img src="img/tut4_fig12.png" alt="drawing" width="500"/>

Depicted in the right side of this last figure is the partial tensor trace of $T$ with its conjugate network. Draw the simplified network that results from using properties of the canonical form to cancel tensors wherever possible.

**(d) Solution**: Canonical form implies that the branch to the left of the $A-D$ link collapses to the identity with its conjugate, so the network simplifies to:

<img src="img/tut4_d_solution.png" alt="drawing" width="400"/>